In [2]:
pip install torch torchvision torchaudio


  Using cached torchaudio-2.3.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00a 0:00:01
Using cached torchaudio-2.3.1-cp310-cp310-macosx_11_0_arm64.whl (1.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install ipywidgets --upgrade
pip install jupyter --upgrade


In [3]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import T5EncoderModel
from diffusers import StableDiffusion3Pipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Load environment variables from a .env file
load_dotenv()

# Log in to Hugging Face Hub
login(token=os.getenv("HF_TOKEN"))

# Check if MPS is available and set the appropriate device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load the text encoder model without quantization
text_encoder = T5EncoderModel.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    subfolder="text_encoder_3",
).to(device)

# Load the Stable Diffusion pipeline with the default text encoder
pipe_default = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    torch_dtype=torch.float32  # Use float32 for CPU and MPS
).to(device)

# Load the Stable Diffusion pipeline with the custom text encoder
pipe_custom = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    text_encoder_3=text_encoder,
    torch_dtype=torch.float32  # Use float32 for CPU and MPS
).to(device)

# Define the prompt and generation parameters
prompt = "a photo of a cat holding a sign that says hello world"
negative_prompt = ""
num_inference_steps = 28
height = 1024
width = 1024
guidance_scale = 7.0

# Generate an image with the default text encoder
image_default = pipe_default(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Generate an image with the custom text encoder
image_custom = pipe_custom(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Save the images
image_default_path = "sd3_hello_world_default.png"
image_custom_path = "sd3_hello_world_custom.png"
image_default.save(image_default_path)
image_custom.save(image_custom_path)

# Display the images in the Jupyter notebook
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

# Display default text encoder image
axs[0].imshow(image_default)
axs[0].axis('off')
axs[0].set_title("Default Text Encoder")

# Display custom text encoder image
axs[1].imshow(image_custom)
axs[1].axis('off')
axs[1].set_title("Custom Text Encoder")

plt.show()


/Users/nick/Code/SF-T5-Example/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/nick/Code/SF-T5-Example/.venv/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/nick/.cache/huggingface/token
Login successful


Loading pipeline components...:  78%|███████▊  | 7/9 [00:15<00:04,  2.23s/it]


ValueError: Cannot instantiate this tokenizer from a slow version. If it's based on sentencepiece, make sure you have sentencepiece installed.